# Imports

In [2]:
# Automatically reloads newest versions of imports if functions in any python files used are changed
# %autoreload

import pandas as pd
import numpy as np
from scipy.interpolate import interp1d
from sklearn.linear_model import LinearRegression


# Load datasets

In [3]:
X_train_estimated_a = pd.read_parquet('../data/A/X_train_estimated.parquet')
X_train_estimated_b = pd.read_parquet('../data/B/X_train_estimated.parquet')
X_train_estimated_c = pd.read_parquet('../data/C/X_train_estimated.parquet')

X_train_observed_a = pd.read_parquet('../data/A/X_train_observed.parquet')
X_train_observed_b = pd.read_parquet('../data/B/X_train_observed.parquet')
X_train_observed_c = pd.read_parquet('../data/C/X_train_observed.parquet')

X_test_estimated_a = pd.read_parquet('../data/A/X_test_estimated.parquet')
X_test_estimated_b = pd.read_parquet('../data/B/X_test_estimated.parquet')
X_test_estimated_c = pd.read_parquet('../data/C/X_test_estimated.parquet')

train_targets_a = pd.read_parquet('../data/A/train_targets.parquet')
train_targets_b = pd.read_parquet('../data/B/train_targets.parquet')
train_targets_c = pd.read_parquet('../data/C/train_targets.parquet')

# Data clean up

In [ ]:
# Combine the datasets 'a', 'b', and 'c'
if 'date_calc' in X_train_estimated_a.columns:
  X_train_estimated_a.drop(columns="date_calc", inplace=True)

X_train = pd.concat([X_train_observed_a, X_train_estimated_a])
# X_train
print(X_train_estimated_a.shape)
print(X_train_observed_a.shape)
print(train_targets_a.shape)
print(X_train.index.shape)
agg_func = {col: 'mean' for col in X_train.columns[1:]}


# Group the rows into blocks of 4 and apply the aggregation function
X_train_downscaled = X_train.groupby(X_train.index // 4).agg({**{'date_forecast': 'first'}, **agg_func})
# Reset the index to make it a regular DataFrame with a new index
X_train_downscaled.reset_index(drop=True, inplace=True)
combined_data = pd.merge(X_train_downscaled, train_targets_a, left_on='date_forecast', right_on='time')
y_train = combined_data['pv_measurement']
combined_data.drop(columns="ceiling_height_agl:m", inplace=True)

# if 'date_forecast' and 'time' and 'pv_measurement' in combined_data.columns:
  # combined_data.drop(columns="date_forecast", inplace=True)
  # combined_data.drop(columns="time", inplace=True)
  # combined_data.drop(columns="pv_measurement", inplace=True)

# x_train = combined_data.dropna(inplace=True)
print(combined_data.shape)
print(y_train)

# Feature engineering

In [ ]:
# Do feature selection etc.

# Training the model

In [35]:

# model_a = LinearRegression()
# model_a.fit(x_train, y_train)

(17576, 46)
(118669, 46)
(34085, 2)
(136245,)
(29667, 47)
0         0.00
1         0.00
2         0.00
3         0.00
4        19.36
         ...  
29662     0.00
29663     0.00
29664     0.00
29665     0.00
29666     0.00
Name: pv_measurement, Length: 29667, dtype: float64


# Make predictions

In [ ]:
y_pred = model_a.predict(X_test_estimated_a)

# Create submission

In [ ]:
y_test_pred = y_pred

test = pd.read_csv('../data/test.csv')
test['prediction'] = y_test_pred
sample_submission = pd.read_csv('sample_submission.csv')
submission = sample_submission[['id']].merge(test[['id', 'prediction']], on='id', how='left')
submission.to_csv('submission.csv', index=False)